In [1]:
import requests
import pandas as pd
import time
import math
from pathlib import Path


In [ ]:
 pip install pytrends

In [ ]:
from pytrends.request import TrendReq

# Inicializa pytrends
pytrends = TrendReq(hl='es-ES', tz=360)

# Define las palabras clave y el período de tiempo
kw_list = ['Taylor Swift']
timeframe = 'today 12-m'

# Realiza la búsqueda
pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='ES')

# Obtiene los datos de interés por región (comunidad autónoma)
interest_by_region = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False)

# Calcula el porcentaje de popularidad de cada comunidad autónoma
total_interest = interest_by_region.sum()
popularity_percentage = (interest_by_region / total_interest) * 100

print(popularity_percentage)

                        Taylor Swift
geoName                             
Andalucía                   5.306799
Aragón                      4.643449
Canarias                    5.389718
Cantabria                   5.306799
Castilla y León             5.306799
Castilla-La Mancha          4.477612
Cataluña                    6.467662
Ceuta                       4.145937
Comunidad Valenciana        5.555556
Comunidad de Madrid         8.291874
Extremadura                 4.643449
Galicia                     6.301824
Islas Baleares              5.555556
La Rioja                    4.892206
Melilla                     3.482587
Navarra                     4.560531
País Vasco                  5.472637
Principado de Asturias      5.472637
Región de Murcia            4.726368


In [2]:
pag = 0
url = 'https://app.ticketmaster.com/discovery/v2/events?apikey=4uZ7cKblFpFckdrfHMGrT2coHBKCiAjs&countryCode=ES&classificationName=music&locale=*&page=' + str(pag)
r = requests.get(url)

if r.status_code == 200:
  respuesta = r.json()
  total_pages = respuesta['page']['totalPages']
  print(total_pages)
  df_datos = pd.DataFrame(respuesta['_embedded']['events'])
  pag += 1
  while(pag < total_pages):
    url = 'https://app.ticketmaster.com/discovery/v2/events?apikey=4uZ7cKblFpFckdrfHMGrT2coHBKCiAjs&countryCode=ES&classificationName=music&locale=*&page=' + str(pag)
    r = requests.get(url)
    res = r.json()
    df_datos_mas = pd.DataFrame(res['_embedded']['events'])
    df_datos = pd.concat([df_datos, df_datos_mas], ignore_index=True)
    pag += 1
else:
  print('Error: ' + r.status_code)
  print(r.text)


len(df_datos)

39


780

In [3]:
df_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             780 non-null    object
 1   type             780 non-null    object
 2   id               780 non-null    object
 3   test             780 non-null    bool  
 4   url              780 non-null    object
 5   locale           780 non-null    object
 6   images           780 non-null    object
 7   sales            780 non-null    object
 8   dates            780 non-null    object
 9   classifications  780 non-null    object
 10  promoter         779 non-null    object
 11  promoters        779 non-null    object
 12  priceRanges      771 non-null    object
 13  seatmap          84 non-null     object
 14  _links           780 non-null    object
 15  _embedded        780 non-null    object
dtypes: bool(1), object(15)
memory usage: 92.3+ KB


In [4]:
print(df_datos['type'].unique())    # la columna type toma únicamente el valor 'event' -> eliminar
df_datos_menos_col = df_datos.drop(columns = ['test', 'url','images',
                                              'promoters', 'type', 'locale',
                                              '_links', 'seatmap'])
df_datos_menos_col

['event']


,name,id,sales,dates,classifications,promoter,priceRanges,_embedded
0,Sunsetland Festival,Z698xZ2qZaa_L,{'public': {'startDateTime': '2024-01-25T11:00...,"{'start': {'localDate': '2024-06-28', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6285', 'name': 'Taborodri Hostelería S...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Recinto Ferial La Fica',..."
1,Servicio de Autobús - BRESH - Grand Opening -...,Z698xZ2qZaa_M,{'public': {'startDateTime': '2024-01-24T14:00...,"{'start': {'localDate': '2024-06-28', 'dateTBD...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'ServicioBus ConcertMusic...
2,BRESH - Grand Opening Concert Music Festival,Z698xZ2qZaa_S,{'public': {'startDateTime': '2024-01-24T14:00...,"{'start': {'localDate': '2024-06-28', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Concert Music Festival',..."
3,Manolo García - Concert Music Festival,Z698xZ2qZaaC4,{'public': {'startDateTime': '2023-11-30T11:00...,"{'start': {'localDate': '2024-06-29', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Concert Music Festival',..."
4,Servicio de Autobús - Manolo García - Concert...,Z698xZ2qZaaC9,{'public': {'startDateTime': '2023-11-30T11:00...,"{'start': {'localDate': '2024-06-29', 'dateTBD...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'ServicioBus ConcertMusic...
...,...,...,...,...,...,...,...,...
775,"The Music of El Señor de los Anillos,El Hobbit...",Z698xZ2qZaFWs,{'public': {'startDateTime': '2023-08-09T15:28...,{'access': {'startDateTime': '2025-03-02T14:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Palacio Vistalegre', 'ty..."
776,The Magical Music of Harry Potter,Z698xZ2qZaaoh,{'public': {'startDateTime': '2023-12-07T16:17...,{'access': {'startDateTime': '2025-03-06T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'Palau de Congressos de T...
777,Barcelona Bar Showroom,Z698xZ2qZaar2,{'public': {'startDateTime': '2024-02-13T16:26...,"{'start': {'localDate': '2024-03-17', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6341', 'name': 'BN GRUP 2019 S.L.'}","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Hyatt Regency Barcelona ...
778,"ATRÉVETE, el musical",Z698xZ2qZaaNL,{'public': {'startDateTime': '2024-02-21T13:30...,{'access': {'startDateTime': '2024-04-27T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5785', 'name': 'Producciones Comediart...","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Casa de la Cultura de Mu...


In [ ]:
df_datos_menos_col.iloc[2]['classifications']

[{'primary': True,
  'segment': {'id': 'KZFzniwnSyZfZ7v7nn', 'name': 'Film'},
  'genre': {'id': 'KnvZfZ7vAkJ', 'name': 'Music'},
  'subGenre': {'id': 'KZazBEonSMnZfZ7vave', 'name': 'Music'},
  'family': False}]

In [5]:
# Definir una función para extraer el valor startDateTime de cada fila
def obtener_start_datetime(fila):
    return fila['dates']['access']['startDateTime']

df_datos_menos_col.loc[0, 'startDateTime']= obtener_start_datetime(df_datos_menos_col.iloc[0])
df_datos_menos_col


KeyError: 'access'

In [ ]:

# Configura tu clave de API de Last.fm
LASTFM_API_KEY = "d3668e7b9ace955aaefafa6e262386ba"
ARTIST_NAME = "Taylor Swift"
DATE = "04-02-2024"  # Ajusta según sea necesario

# Construye la URL para solicitar datos de Last.fm
url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={ARTIST_NAME}&api_key={LASTFM_API_KEY}&format=json"

# Realiza la solicitud a la API de Last.fm
response = requests.get(url)
data = response.json()

# Extrae el número de oyentes en la fecha específica (si está disponible en la respuesta)
if "artist" in data and "stats" in data["artist"]:
    listeners = data["artist"]["stats"]["listeners"]
    print(f"Número de oyentes de {ARTIST_NAME} en {DATE}: {listeners}")
else:
    print(f"No se encontró información para {ARTIST_NAME} en la fecha {DATE}.")


Número de oyentes de Taylor Swift en 04-02-2024: 4495452


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # Nombre del artista que deseas buscar
    artist_name = 'Taylor Swift'

    # URL de la API de Spotify para buscar al artista
    search_url = 'https://api.spotify.com/v1/search'
    search_params = {
        'q': artist_name,
        'type': 'artist',
    }

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud de búsqueda del artista
    search_response = requests.get(search_url, params=search_params, headers=headers)
    search_data = search_response.json()

    # Obtener el ID del primer resultado (artista)
    if 'artists' in search_data and 'items' in search_data['artists'] and len(search_data['artists']['items']) > 0:
        artist_id = search_data['artists']['items'][0]['id']
        print(f'El ID del artista {artist_name} es: {artist_id}')

    else:
        print(f'No se encontró al artista {artist_name}')

else:
    print('Error al obtener el token de acceso')
    print(token_data)


El ID del artista Taylor Swift es: 06HL4z0CvFAxyc27GXpf02


In [ ]:
import requests

# Tu API key y secret obtenidos en el paso anterior
client_id = '3e082f0f2dd240c1beb66c9705a663a5'
client_secret = 'd3dd1f7886eb4ba4aa5a76c9095120d7'

# URL de la API de Spotify para obtener un token de acceso
token_url = 'https://accounts.spotify.com/api/token'

# Parámetros necesarios para obtener el token de acceso
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

# Obtener el token de acceso
token_response = requests.post(token_url, data=token_params)
token_data = token_response.json()

# Verificar si se obtuvo correctamente el token
if 'access_token' in token_data:
    access_token = token_data['access_token']

    # ID del artista que deseas consultar (puedes obtenerlo de la API de búsqueda)
    artist_id = '06HL4z0CvFAxyc27GXpf02'

    # URL de la API de Spotify para obtener información del artista
    artist_url = f'https://api.spotify.com/v1/artists/{artist_id}'

    # Configurar la cabecera con el token de acceso
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Hacer la solicitud para obtener información del artista
    artist_response = requests.get(artist_url, headers=headers)
    artist_data = artist_response.json()

    # Verificar si se obtuvo correctamente la información del artista
    if 'followers' in artist_data:
        followers_count = artist_data['followers']['total']
        print(f'El artista tiene {followers_count} oyentes mensuales.')

    else:
        print('Error al obtener información del artista')
        print(artist_data)

else:
    print('Error al obtener el token de acceso')
    print(token_data)


El artista tiene 101484352 oyentes mensuales.


In [ ]:
path = Path.cwd()   # 21/2/24 -> 39 paginas - 772 filas

print(path)
p = Path(path,"datos_21_02.csv")
df_datos.to_csv(p,index=False)
print(p)

/content
/content/datos_21_02.csv


In [6]:
df_datos_menos_col

,name,id,sales,dates,classifications,promoter,priceRanges,_embedded
0,Sunsetland Festival,Z698xZ2qZaa_L,{'public': {'startDateTime': '2024-01-25T11:00...,"{'start': {'localDate': '2024-06-28', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6285', 'name': 'Taborodri Hostelería S...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Recinto Ferial La Fica',..."
1,Servicio de Autobús - BRESH - Grand Opening -...,Z698xZ2qZaa_M,{'public': {'startDateTime': '2024-01-24T14:00...,"{'start': {'localDate': '2024-06-28', 'dateTBD...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'ServicioBus ConcertMusic...
2,BRESH - Grand Opening Concert Music Festival,Z698xZ2qZaa_S,{'public': {'startDateTime': '2024-01-24T14:00...,"{'start': {'localDate': '2024-06-28', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Concert Music Festival',..."
3,Manolo García - Concert Music Festival,Z698xZ2qZaaC4,{'public': {'startDateTime': '2023-11-30T11:00...,"{'start': {'localDate': '2024-06-29', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Concert Music Festival',..."
4,Servicio de Autobús - Manolo García - Concert...,Z698xZ2qZaaC9,{'public': {'startDateTime': '2023-11-30T11:00...,"{'start': {'localDate': '2024-06-29', 'dateTBD...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5875', 'name': 'CONCERT TOUR Y EN VIVO...","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'ServicioBus ConcertMusic...
...,...,...,...,...,...,...,...,...
775,"The Music of El Señor de los Anillos,El Hobbit...",Z698xZ2qZaFWs,{'public': {'startDateTime': '2023-08-09T15:28...,{'access': {'startDateTime': '2025-03-02T14:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...","{'venues': [{'name': 'Palacio Vistalegre', 'ty..."
776,The Magical Music of Harry Potter,Z698xZ2qZaaoh,{'public': {'startDateTime': '2023-12-07T16:17...,{'access': {'startDateTime': '2025-03-06T19:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5813', 'name': 'Star Entertainment GmbH'}","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'Palau de Congressos de T...
777,Barcelona Bar Showroom,Z698xZ2qZaar2,{'public': {'startDateTime': '2024-02-13T16:26...,"{'start': {'localDate': '2024-03-17', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6341', 'name': 'BN GRUP 2019 S.L.'}","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Hyatt Regency Barcelona ...
778,"ATRÉVETE, el musical",Z698xZ2qZaaNL,{'public': {'startDateTime': '2024-02-21T13:30...,{'access': {'startDateTime': '2024-04-27T20:00...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '5785', 'name': 'Producciones Comediart...","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Casa de la Cultura de Mu...


In [10]:
def obtener_min_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        return fila['priceRanges'][0]['min']
    else:
        return None

def obtener_max_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        return fila['priceRanges'][0]['max']
    else:
        return None
# Aplicar las funciones a cada fila de la columna 'prices' y asignar los resultados a nuevas columnas 'min_price' y 'max_price'
df_datos_menos_col['min_price'] = df_datos_menos_col.apply(obtener_min_price, axis=1)
df_datos_menos_col['max_price'] = df_datos_menos_col.apply(obtener_max_price, axis=1)

def venue_name(fila):
    return fila['_embedded']['venues'][0]['name']

def obtener_start_datetime(fila):
        return fila['sales']['public'].get('startDateTime')

def obtener_end_datetime(fila):
    return fila['sales']['public'].get('endDateTime')

df_datos_menos_col.iloc[0]['_embedded']

df_datos_menos_col['StartTimeSale'] = df_datos_menos_col.apply(obtener_start_datetime, axis=1)
df_datos_menos_col['EndTimeSale'] = df_datos_menos_col.apply(obtener_end_datetime, axis=1)

df_datos_menos_col['VenueName'] = df_datos_menos_col.apply(venue_name, axis=1)

# #print(df_datos_menos_col.iloc[1]['priceRanges'])
#df_datos_menos_col.iloc[0]['sales']
df_datos_menos_col.iloc[2]['_embedded']
#df_datos_menos_col

{'venues': [{'name': 'Concert Music Festival',
   'type': 'venue',
   'id': 'Z598xZ2qZdkaF',
   'test': False,
   'url': 'https://www.ticketmaster.es/venue/concert-music-festival-chiclana-de-la-frontera-entradas/rcsancpchi/114',
   'locale': 'en-us',
   'postalCode': '11139',
   'timezone': 'Europe/Madrid',
   'city': {'name': 'Chiclana de la Frontera'},
   'state': {'name': 'Cádiz'},
   'country': {'name': 'Spain', 'countryCode': 'ES'},
   'address': {'line1': 'Calle Avenida (Poblado Sancti Petri)'},
   'location': {'longitude': '-6.20735', 'latitude': '36.39397'},
   'upcomingEvents': {'mfx-es': 35, '_total': 35, '_filtered': 0},
   '_links': {'self': {'href': '/discovery/v2/venues/Z598xZ2qZdkaF?locale=en-us'}}}],
 'attractions': [{'name': 'Bresh',
   'type': 'attraction',
   'id': 'K8vZ917_4H0',
   'test': False,
   'url': 'https://www.ticketmaster.ie/bresh-tickets/artist/5349904',
   'locale': 'en-us',
   'images': [{'ratio': '16_9',
     'url': 'https://s1.ticketm.net/dam/c/80b/f3

In [60]:
def venue_name(fila):
    return fila['_embedded']['venues'][0]['name']

def venue_city(fila):
  return fila['_embedded']['venues'][0]['city']['name']

def venue_state(fila):
  return fila['_embedded']['venues'][0]['state']['name']

def venue_country(fila):
  return fila['_embedded']['venues'][0]['country']['name']

df_datos_menos_col['VenueName'] = df_datos_menos_col.apply(venue_name, axis=1)
df_datos_menos_col['VenueCity'] = df_datos_menos_col.apply(venue_city, axis=1)
df_datos_menos_col['VenueState'] = df_datos_menos_col.apply(venue_state, axis=1)
df_datos_menos_col['VenueCountry'] = df_datos_menos_col.apply(venue_country, axis=1)

df_datos_menos_col.iloc[0]['_embedded']

{'venues': [{'name': 'Recinto Ferial La Fica',
   'type': 'venue',
   'id': 'Z598xZ2qZed7e',
   'test': False,
   'url': 'https://www.ticketmaster.es/venue/recinto-ferial-la-fica-murcia-entradas/rflaficmur/114',
   'locale': 'en-us',
   'postalCode': '30003',
   'timezone': 'Europe/Madrid',
   'city': {'name': 'Murcia'},
   'state': {'name': 'Murcia'},
   'country': {'name': 'Spain', 'countryCode': 'ES'},
   'address': {'line1': 'Diseminado Primero Mayo, 19'},
   'location': {'longitude': '-1.11297', 'latitude': '37.9847'},
   'upcomingEvents': {'mfx-es': 1, '_total': 1, '_filtered': 0},
   '_links': {'self': {'href': '/discovery/v2/venues/Z598xZ2qZed7e?locale=en-us'}}}],
 'attractions': [{'name': 'Sunsetland Festival',
   'type': 'attraction',
   'id': 'K8vZ917bGC7',
   'test': False,
   'url': 'https://www.ticketmaster.ca/sunsetland-festival-tickets/artist/2656599',
   'locale': 'en-us',
   'images': [{'ratio': '16_9',
     'url': 'https://s1.ticketm.net/dam/a/11c/c57cef94-e038-4b1a-

In [20]:
taylor = df_datos[df_datos['name'].str.contains('Taylor Swift')]
taylor = taylor.drop(columns = ['test', 'url','images',
                                              'promoters', 'type', 'locale',
                                              '_links', 'seatmap'])


def obtener_min_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['min']
    return None

def obtener_max_price(fila):
    if 'priceRanges' in fila and isinstance(fila['priceRanges'], list) and fila['priceRanges']:
        for price_range in fila['priceRanges']:
            if price_range.get('type') == 'standard':
                return price_range['max']
    return None

# Aplicar las funciones a cada fila de la columna 'prices' y asignar los resultados a nuevas columnas 'min_price' y 'max_price'
taylor['min_price'] = taylor.apply(obtener_min_price, axis=1)
taylor['max_price'] = taylor.apply(obtener_max_price, axis=1)

taylor['StartTimeSale'] = taylor.apply(obtener_start_datetime, axis=1)
taylor['EndTimeSale'] = taylor.apply(obtener_end_datetime, axis=1)

taylor['VenueName'] = taylor.apply(venue_name, axis=1)
taylor['VenueCity'] = taylor.apply(venue_city, axis=1)
taylor['VenueState'] = taylor.apply(venue_state, axis=1)
taylor['VenueCountry'] = taylor.apply(venue_country, axis=1)

print(taylor.iloc[0]['priceRanges'])
print(taylor.iloc[1]['priceRanges'])
taylor

[{'type': 'standard including fees', 'currency': 'EUR', 'min': 282.0, 'max': 589.0}, {'type': 'standard', 'currency': 'EUR', 'min': 249.0, 'max': 549.0}]
[{'type': 'standard', 'currency': 'EUR', 'min': 65.0, 'max': 200.0}, {'type': 'standard including fees', 'currency': 'EUR', 'min': 73.5, 'max': 226.5}]


,name,id,sales,dates,classifications,promoter,priceRanges,_embedded,min_price,max_price,StartTimeSale,EndTimeSale,VenueName,VenueCity,VenueState,VenueCountry
262,Taylor Swift | The Eras Tour - VIP Packages,Z698xZ2qZaFyD,{'public': {'startDateTime': '2023-07-13T12:00...,"{'start': {'localDate': '2024-05-30', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6165', 'name': 'LAST TOUR EUROPA, SL'}","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Estadio Santiago Bernabé...,249.0,549.0,2023-07-13T12:00:00Z,2024-05-30T16:30:00Z,Estadio Santiago Bernabéu,Madrid,Madrid,Spain
263,Taylor Swift | The Eras Tour,Z698xZ2qZaFyq,{'public': {'startDateTime': '2023-07-13T12:00...,{'access': {'startDateTime': '2024-05-30T16:30...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6165', 'name': 'LAST TOUR EUROPA, SL'}","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'Estadio Santiago Bernabé...,65.0,200.0,2023-07-13T12:00:00Z,2024-05-30T16:30:00Z,Estadio Santiago Bernabéu,Madrid,Madrid,Spain


In [57]:
taylor.iloc[0]['_embedded']['attractions'][0]['classifications']

[{'primary': True,
  'segment': {'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'},
  'genre': {'id': 'KnvZfZ7vAev', 'name': 'Pop'},
  'subGenre': {'id': 'KZazBEonSMnZfZ7vk1t', 'name': 'Pop'},
  'type': {'id': 'KZAyXgnZfZ7v7nI', 'name': 'Undefined'},
  'subType': {'id': 'KZFzBErXgnZfZ7v7lJ', 'name': 'Undefined'},
  'family': False}]

In [61]:
taylor.iloc[0]['_embedded']['attractions'][0]['classifications'][0]['genre']['name']

def genre_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['genre']['name']

def subgenre_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['subGenre']['name']

def type_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['type']['name']

def subtype_event(fila):
  return fila['_embedded']['attractions'][0]['classifications'][0]['subType']['name']

taylor['genre'] = taylor.apply(genre_event, axis = 1)
taylor['subGenre'] = taylor.apply(subgenre_event, axis = 1)
taylor['type'] = taylor.apply(type_event, axis = 1)
taylor['subType'] = taylor.apply(subtype_event, axis = 1)

taylor

,name,id,sales,dates,classifications,promoter,priceRanges,_embedded,min_price,max_price,StartTimeSale,EndTimeSale,VenueName,VenueCity,VenueState,VenueCountry,genre,subGenre,type,subType
262,Taylor Swift | The Eras Tour - VIP Packages,Z698xZ2qZaFyD,{'public': {'startDateTime': '2023-07-13T12:00...,"{'start': {'localDate': '2024-05-30', 'localTi...","[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6165', 'name': 'LAST TOUR EUROPA, SL'}","[{'type': 'standard including fees', 'currency...",{'venues': [{'name': 'Estadio Santiago Bernabé...,249.0,549.0,2023-07-13T12:00:00Z,2024-05-30T16:30:00Z,Estadio Santiago Bernabéu,Madrid,Madrid,Spain,Pop,Pop,Undefined,Undefined
263,Taylor Swift | The Eras Tour,Z698xZ2qZaFyq,{'public': {'startDateTime': '2023-07-13T12:00...,{'access': {'startDateTime': '2024-05-30T16:30...,"[{'primary': True, 'segment': {'id': 'KZFzniwn...","{'id': '6165', 'name': 'LAST TOUR EUROPA, SL'}","[{'type': 'standard', 'currency': 'EUR', 'min'...",{'venues': [{'name': 'Estadio Santiago Bernabé...,65.0,200.0,2023-07-13T12:00:00Z,2024-05-30T16:30:00Z,Estadio Santiago Bernabéu,Madrid,Madrid,Spain,Pop,Pop,Undefined,Undefined


In [62]:
taylor.iloc[0]['classifications']

[{'primary': True,
  'segment': {'id': 'KZFzniwnSyZfZ7v7nJ', 'name': 'Music'},
  'genre': {'id': 'KnvZfZ7vAeA', 'name': 'Rock'},
  'subGenre': {'id': 'KZazBEonSMnZfZ7v6F1', 'name': 'Pop'},
  'family': False}]